In [2]:
import pandas as pd
import re
pd.set_option('display.max_columns', 90)
pd.set_option('display.max_colwidth', None)
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [3]:
df = pd.read_feather('../datasets/orders.feather')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1191861 entries, 0 to 437216
Data columns (total 38 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   id                       1191861 non-null  int32  
 1   order_date               1191861 non-null  object 
 2   subject_id               1181529 non-null  float32
 3   purpose                  1025351 non-null  object 
 4   lesson_price             1191861 non-null  int32  
 5   lesson_duration          1191861 non-null  int32  
 6   lesson_place             31 non-null       object 
 7   home_metro_id            642447 non-null   float32
 8   add_info                 1044688 non-null  object 
 9   start_date               705578 non-null   object 
 10  working_teacher_id       705988 non-null   float32
 11  status_id                1191861 non-null  int32  
 12  comments                 341468 non-null   object 
 13  prefered_teacher_id      4 non-null        float

In [5]:
duplicates = {}

# Перебор всех колонок для сравнения
for i, col1 in enumerate(df.columns):
    for col2 in df.columns[i + 1:]:
        # Проверка на идентичность данных в колонках
        if df[col1].equals(df[col2]):
            # Запись найденных дубликатов
            duplicates.setdefault(col1, []).append(col2)

# Вывод результатов
if duplicates:
    for key, values in duplicates.items():
        print(f"Колонка '{key}' дублируется в колонках: {values}")
else:
    print("Дублирующихся колонок не найдено.")

Дублирующихся колонок не найдено.


In [6]:
# Вычисляем процент пропущенных данных для каждой колонки
percent_missing = df.isnull().mean() * 100

# Определяем колонки для удаления
cols_to_drop = percent_missing[percent_missing > 85].index

# Удаляем эти колонки
df.drop(cols_to_drop, axis=1, inplace=True)

In [7]:
df['amount_to_pay'] = df['amount_to_pay'].str.replace(',', '.').astype(float)

In [8]:
df = df[df['lesson_price'] <= 5000]
df = df[df['lesson_duration'] <= 120]
df = df[df['minimal_price'] <= 5000]
df = df[df['teacher_experience_to'] <= 75]

In [9]:
df['order_date'] = pd.to_datetime(df['order_date'])
df['start_date'] = pd.to_datetime(df['start_date'])
df['estimated_fee'] = pd.to_datetime(df['estimated_fee'], errors='coerce')

C:\Users\stone\AppData\Local\Temp\ipykernel_7216\4248979078.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['estimated_fee'] = pd.to_datetime(df['estimated_fee'], errors='coerce')


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1183000 entries, 0 to 437215
Data columns (total 33 columns):
 #   Column                   Non-Null Count    Dtype         
---  ------                   --------------    -----         
 0   id                       1183000 non-null  int32         
 1   order_date               1183000 non-null  datetime64[ns]
 2   subject_id               1172671 non-null  float32       
 3   purpose                  1019444 non-null  object        
 4   lesson_price             1183000 non-null  int32         
 5   lesson_duration          1183000 non-null  int32         
 6   home_metro_id            637493 non-null   float32       
 7   add_info                 1038749 non-null  object        
 8   start_date               701257 non-null   datetime64[ns]
 9   working_teacher_id       701666 non-null   float32       
 10  status_id                1183000 non-null  int32         
 11  comments                 336397 non-null   object        
 12  amount

In [11]:
df['purpose']

0                                                                                                                        None
1                                                                                                                        None
2               Разделы: начертательная геометрия.\nКатегория ученика: студентка 1 курса.\nЦель занятий: помощь по программе.
3                Разделы:  школьный курс.\nКатегория ученика: школьник 9 класса.\nЦель занятий: помощь по школьной программе.
4                                                                                                                        None
                                                                 ...                                                         
437211                                         Дополнения: ОГЭ (ГИА), подготовка к олимпиадам.\nКатегория ученика: 9 класс.\n
437212    Разделы:  школьный курс.\nКатегория ученика:  два школьника 7 класса.\nЦель занятий: занятия по школьной про

In [12]:
df

,id,order_date,subject_id,purpose,lesson_price,lesson_duration,home_metro_id,add_info,start_date,working_teacher_id,status_id,comments,amount_to_pay,planned_lesson_number,creator_id,pupil_category_new_id,lessons_per_week,minimal_price,teacher_sex,teacher_experience_from,teacher_experience_to,lesson_place_new,pupil_knowledgelvl,teacher_age_from,teacher_age_to,chosen_teachers_only,no_teachers_available,source_id,original_order_id,client_id,additional_status_id,estimated_fee,is_display_to_teachers
0,2294054,2022-11-01 00:01:38.237,1.0,None,0,60,NaN,None,NaT,NaN,16,None,0.0,0,NaN,NaN,0,0,3,0,0,0,0,0,0,0,0,18,2294000.0,675960,7.0,NaT,1
1,2294055,2022-11-01 00:03:11.753,1.0,None,0,60,NaN,None,NaT,NaN,7,"\n01.11.2022 11:46 - ""Заявка прикреплена к заявке № 2294000"" (Алешина Марина)",0.0,0,NaN,NaN,0,0,3,0,0,0,0,0,0,0,0,18,2294000.0,675960,7.0,NaT,1
2,2294056,2022-11-01 00:04:01.563,21.0,Разделы: начертательная геометрия.\nКатегория ученика: студентка 1 курса.\nЦель занятий: помощь по программе.,800,60,NaN,"Место проведения занятий: дистанционно (например, Skype). Время МСК.\nПредполагаемое число занятий в неделю: 1, будни и суббота после 17, воскресенье в течение дня.",2022-11-01 10:37:02.567,143480.0,16,None,0.0,7,NaN,14.0,2,0,3,0,0,4,0,0,0,0,0,23,NaN,1024439,10.0,NaT,1
3,2294057,2022-11-01 00:08:47.237,8.0,Разделы: школьный курс.\nКатегория ученика: школьник 9 класса.\nЦель занятий: помощь по школьной программе.,700,60,NaN,"Место проведения занятий: дистанционно (например, Skype).\nПредполагаемое число занятий в неделю: 1-2, вторник, четверг с 17, суббота с 14.\nОпыт репетитора: от 5 лет.\nВозраст репетитора: от 20 лет до 90 лет.\nПол репетитора: не имеет значения.\nСтатус репетитора: школьный преподаватель.",2022-11-01 00:08:47.437,236392.0,16,None,0.0,10,47749.0,10.0,3,0,3,5,0,4,0,20,90,0,0,14,2293804.0,1024340,17.0,NaT,1
4,2294058,2022-11-01 00:11:49.417,8.0,None,0,60,122.0,None,NaT,NaN,16,Тестовая группа : SplitTestA,0.0,0,NaN,NaN,0,0,0,0,0,2,0,0,0,0,0,16,NaN,1024440,5.0,NaT,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437211,2280502,2022-10-24 19:44:19.933,13.0,"Дополнения: ОГЭ (ГИА), подготовка к олимпиадам.\nКатегория ученика: 9 класс.\n",900,60,792.0,None,NaT,NaN,16,None,0.0,10,NaN,10.0,3,0,0,0,0,6,0,23,90,0,0,23,NaN,1019632,5.0,NaT,1
437212,2280503,2022-10-24 19:45:30.277,9.0,Разделы: школьный курс.\nКатегория ученика: два школьника 7 класса.\nЦель занятий: занятия по школьной программе.,1000,60,NaN,"Место проведения занятий: дистанционно (например, Skype). МОСКОВСКОЕ ВРЕМЯ.\nПредполагаемое число занятий в неделю: 1. Вторник - после 16 часов.\nОпыт репетитора: от 1 года.\nВозраст репетитора: от 20 лет\nПол репетитора: женский.\nСтатус репетитора: школьный преподаватель, частный преподаватель.",2022-10-24 20:17:29.977,205868.0,6,None,2650.0,10,NaN,8.0,2,800,2,3,10,4,0,21,30,0,0,23,NaN,824363,NaN,NaT,1
437213,2280504,2022-10-24 19:45:40.713,1.0,Разделы: школьный курс.\nКатегория ученика: школьник 5 класса.\nЦель занятий: помощь по школьной программе.,1000,120,NaN,"Место проведения занятий: дистанционно (например, Skype).\nПредполагаемое число занятий в неделю: 1. Воскресение любое время.\nОпыт репетитора: не имеет значения.\nВозраст репетитора: от 18 лет до 45 лет.\nПол репетитора: женский.\nСтатус репетитора: не имеет значения.",2022-10-24 20:07:25.370,171067.0,16,None,0.0,10,983555.0,6.0,2,0,2,0,0,4,0,18,45,0,0,14,2280289.0,362987,13.0,NaT,1
437214,2280505,2022-10-24 19:45:44.477,2.0,Разделы: общий курс.\nКатегория ученика: 1 класс.\nУровень знаний: с нуля.\nОЧНЫЕ ЗАНЯТИЯ.,600,60,489.0,Район: Ленинский район.\nМесто проведения занятий: у преподавателя ( только Ленинский район).\nПредполагаемое число занятий в неделю: 1-2. Выходные дни.\nОпыт репетитора: не имеет значения.\nВозраст репетитора: от 21 года.\nПол репетитора: женский.\nСтатус репетитора: не имеет значения.,2022-10-24 20:45:51.357,215909.0,16,None,0.0,10,NaN,2.0,3,500,2,

In [13]:
df['purpose'].fillna('', inplace=True)

In [14]:
def extract_student_category(text):
    """Функция не идеальна"""
    if text is None:
        return 'unknown'
    patterns = {
        'preschool': r'\b1-3 года\b|\b4-5 лет\b|\b6-7 лет\b',
        'school': r'\b\d{1,2} класс\b',
        'student': r'\bстудент\b',
        'adult': r'\bвзрослый\b'
    }
    for category, pattern in patterns.items():
        if re.search(pattern, text, re.IGNORECASE):
            return category
    return 'unknown' 

#def extract_subject(text):
#    if text is None:
#        return None
#    match = re.search(r'Разделы: ([^.]+)', text)
#    return match.group(1).strip() if match else None

#def extract_additions(text):
#    if text is None:
#        return None
#    match = re.search(r'Дополнения: ([^.]+)', text)
#    return match.group(1).strip() if match else None

def extract_info(row):
    row['student_category'] = extract_student_category(row['purpose'])
    #row['subject'] = extract_subject(row['purpose'])
    #row['additions'] = extract_additions(row['purpose'])
    return row

**def extract_subject(text)** - 11 тысяч признаков(понимаю чт оне все коректны и есть повторения,можно было бы срезать кол-во символов в строке,но кому то нужны например все 3 предмета,так что сюда подойдет вариант который я напишу ниже)

Достает супер много признаков,перепишу функцию,что бы создать хотя бы 1 значимый класс(Экзамены)

Рекомендую заказчику обратить на это внимание,решение проблемы вижу таким

либо вы даете список всех разделов Математика,русский,информатика и др. далее создаем из них классы и делаем разметку данных,затем отдаем условному BERT, но я бы выбрал Spacy и выделяем именнованные сущности и кодируем их  в классы

У нас получается отдельный столбец классов 

Либо мы все размечаем руками к прмиеру через докано

для **def extract_additions(text)**(1200 признаков) история та же, только тут будут более детальные указания, ЕГЕ ОГЭ универ и т.д.


Что нам это дает?

Модели будет проще найти зависимости в мотивации заказчика,к примеру подготвка к ЕГЭ по математике как минимум (хотя бы в половине случаях) будет стоять в приоритете ,чем игра на гитаре(условно)

Следовательно заказчик с большей вроятностью оплатит матан чем гитару

В случае с этой функции могут быть законости тоже - можно достать информацию к примеру такую(Опять же условно ему нужен реп по гитаре,потому что он сдает экзамен в музыкалке,а второму - для хобби)

Получается опять факт оплаты и первого пользователя гораздо выше,если реппетитор соответствует ожиданиям заказчика

Далее,примите еще 1 факт во внимание,так как у нас подовляющее большинство либо школьники(без денег),либо студенты(то же без денег) - то оплату производят их опекуны,РОДИТЕЛЯМ ВАЖНО что бы ребенок СДАЛ ЭКЗАМЕН а не научился играть на гитаре(УСЛОВНО)

Применить первый способ я не могу так как у меня нет АКТУАЛЬНОГО и ПОЛНОГО списка предметов - в реализации он не очень то и сложен

Второй способ невероятно времязатратен,размечать каждую строчку данных из целлого миллиона -это очень долго и трудоёмко(у меня есть в этом опыт,я знаю о чём говорю - одному это не посилу даже за месяц сделать)

Так же и для другой функции класс заказчика

Функции которые я рассмотрел напишу нижу,но в Идеале нужна полноценная консултация с заказчиком проекта,так как текс не совсем четко структурирован


In [15]:
#ВСЕ ОНИ ДОСТАЮТ ОЧЕНЬ МНОГО ШУМА И НЕ СТРУКТУРИРОВАННОГО ТЕКСТА
# Требуется реализация через NLP


#def extract_knowledge_level(text):
#    match = re.search(r'Уровень знаний: ([^.]+)', text)
#    return match.group(1).strip() if match else None
#
#def extract_study_goal(text):
#    match = re.search(r'Цель занятий: ([^.]+)', text)
#    return match.group(1).strip() if match else None
#
#def extract_exam_date_score(text):
#    exam_date = re.search(r'Предполагаемая дата экзамена: ([^.\n]+)', text)
#    desired_score = re.search(r'Желаемый балл на экзамене: (\d+)', text)
#    current_score = re.search(r'Текущий балл на экзамене: (\d+)', text)
#    return {
#        'exam_date': exam_date.group(1).strip() if exam_date else None,
#        'desired_score': int(desired_score.group(1)) if desired_score else None,
#        'current_score': int(current_score.group(1)) if current_score else None
#    }


In [16]:
df

,id,order_date,subject_id,purpose,lesson_price,lesson_duration,home_metro_id,add_info,start_date,working_teacher_id,status_id,comments,amount_to_pay,planned_lesson_number,creator_id,pupil_category_new_id,lessons_per_week,minimal_price,teacher_sex,teacher_experience_from,teacher_experience_to,lesson_place_new,pupil_knowledgelvl,teacher_age_from,teacher_age_to,chosen_teachers_only,no_teachers_available,source_id,original_order_id,client_id,additional_status_id,estimated_fee,is_display_to_teachers
0,2294054,2022-11-01 00:01:38.237,1.0,,0,60,NaN,None,NaT,NaN,16,None,0.0,0,NaN,NaN,0,0,3,0,0,0,0,0,0,0,0,18,2294000.0,675960,7.0,NaT,1
1,2294055,2022-11-01 00:03:11.753,1.0,,0,60,NaN,None,NaT,NaN,7,"\n01.11.2022 11:46 - ""Заявка прикреплена к заявке № 2294000"" (Алешина Марина)",0.0,0,NaN,NaN,0,0,3,0,0,0,0,0,0,0,0,18,2294000.0,675960,7.0,NaT,1
2,2294056,2022-11-01 00:04:01.563,21.0,Разделы: начертательная геометрия.\nКатегория ученика: студентка 1 курса.\nЦель занятий: помощь по программе.,800,60,NaN,"Место проведения занятий: дистанционно (например, Skype). Время МСК.\nПредполагаемое число занятий в неделю: 1, будни и суббота после 17, воскресенье в течение дня.",2022-11-01 10:37:02.567,143480.0,16,None,0.0,7,NaN,14.0,2,0,3,0,0,4,0,0,0,0,0,23,NaN,1024439,10.0,NaT,1
3,2294057,2022-11-01 00:08:47.237,8.0,Разделы: школьный курс.\nКатегория ученика: школьник 9 класса.\nЦель занятий: помощь по школьной программе.,700,60,NaN,"Место проведения занятий: дистанционно (например, Skype).\nПредполагаемое число занятий в неделю: 1-2, вторник, четверг с 17, суббота с 14.\nОпыт репетитора: от 5 лет.\nВозраст репетитора: от 20 лет до 90 лет.\nПол репетитора: не имеет значения.\nСтатус репетитора: школьный преподаватель.",2022-11-01 00:08:47.437,236392.0,16,None,0.0,10,47749.0,10.0,3,0,3,5,0,4,0,20,90,0,0,14,2293804.0,1024340,17.0,NaT,1
4,2294058,2022-11-01 00:11:49.417,8.0,,0,60,122.0,None,NaT,NaN,16,Тестовая группа : SplitTestA,0.0,0,NaN,NaN,0,0,0,0,0,2,0,0,0,0,0,16,NaN,1024440,5.0,NaT,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437211,2280502,2022-10-24 19:44:19.933,13.0,"Дополнения: ОГЭ (ГИА), подготовка к олимпиадам.\nКатегория ученика: 9 класс.\n",900,60,792.0,None,NaT,NaN,16,None,0.0,10,NaN,10.0,3,0,0,0,0,6,0,23,90,0,0,23,NaN,1019632,5.0,NaT,1
437212,2280503,2022-10-24 19:45:30.277,9.0,Разделы: школьный курс.\nКатегория ученика: два школьника 7 класса.\nЦель занятий: занятия по школьной программе.,1000,60,NaN,"Место проведения занятий: дистанционно (например, Skype). МОСКОВСКОЕ ВРЕМЯ.\nПредполагаемое число занятий в неделю: 1. Вторник - после 16 часов.\nОпыт репетитора: от 1 года.\nВозраст репетитора: от 20 лет\nПол репетитора: женский.\nСтатус репетитора: школьный преподаватель, частный преподаватель.",2022-10-24 20:17:29.977,205868.0,6,None,2650.0,10,NaN,8.0,2,800,2,3,10,4,0,21,30,0,0,23,NaN,824363,NaN,NaT,1
437213,2280504,2022-10-24 19:45:40.713,1.0,Разделы: школьный курс.\nКатегория ученика: школьник 5 класса.\nЦель занятий: помощь по школьной программе.,1000,120,NaN,"Место проведения занятий: дистанционно (например, Skype).\nПредполагаемое число занятий в неделю: 1. Воскресение любое время.\nОпыт репетитора: не имеет значения.\nВозраст репетитора: от 18 лет до 45 лет.\nПол репетитора: женский.\nСтатус репетитора: не имеет значения.",2022-10-24 20:07:25.370,171067.0,16,None,0.0,10,983555.0,6.0,2,0,2,0,0,4,0,18,45,0,0,14,2280289.0,362987,13.0,NaT,1
437214,2280505,2022-10-24 19:45:44.477,2.0,Разделы: общий курс.\nКатегория ученика: 1 класс.\nУровень знаний: с нуля.\nОЧНЫЕ ЗАНЯТИЯ.,600,60,489.0,Район: Ленинский район.\nМесто проведения занятий: у преподавателя ( только Ленинский район).\nПредполагаемое число занятий в неделю: 1-2. Выходные дни.\nОпыт репетитора: не имеет значения.\nВозраст репетитора: от 21 года.\nПол репетитора: женский.\nСтатус репетитора: не имеет значения.,2022-10-24 20:45:51.357,215909.0,16,None,0.0,10,NaN,2.0,3,500,2,0,0,2,1,21,6

In [17]:
df['purpose'].fillna('', inplace=True)

In [18]:
df = df.apply(extract_info, axis=1)

In [19]:
def extract_exam_preparation(text):
    """экзамен"""
    if text is None:
        return 0
    # Ищем упоминания ЕГЭ, ОГЭ или экзамена как отдельных слов
    if re.search(r'\b(ЕГЭ|ОГЭ|экзамен)\b', text, re.IGNORECASE):
        return 1
    else:
        return 0

# Применяем функцию к столбцу 'purpose' и создаем новый столбец 'exam_preparation'
df['exam_preparation'] = df['purpose'].apply(extract_exam_preparation)

In [20]:
df['exam_preparation'].value_counts()

exam_preparation
0    819657
1    363343
Name: count, dtype: int64

exam_preparation
0    826719
1    365142
Name: count, dtype: int64

Как видите мое предложение по реализации того,что я предложил вышет имеет право на жизнь

Учитываю что я делал через регулярки(я мог где-то накосячить и достать не все значения) мы получили 1/3 от всех закзов которые у нас есть,а это люди с экзаменами(а это может быть лишний копеечкой для вашего бизнеса)

Уверен,что можно достать гораздо больше данных и ОЧЕНЬ РАЗНЫХ

In [21]:
for text in df['add_info'].head(100):
    print(text)
    print('-' * 80) 

None
--------------------------------------------------------------------------------
None
--------------------------------------------------------------------------------
Место проведения занятий: дистанционно (например, Skype). Время МСК.
Предполагаемое число занятий в неделю: 1, будни и суббота после 17, воскресенье в течение дня.
--------------------------------------------------------------------------------
Место проведения занятий: дистанционно (например, Skype).
Предполагаемое число занятий в неделю: 1-2, вторник, четверг с 17, суббота с 14.
Опыт репетитора: от 5 лет.
Возраст репетитора: от 20 лет до 90 лет.
Пол репетитора: не имеет значения.
Статус репетитора: школьный преподаватель.
--------------------------------------------------------------------------------
None
--------------------------------------------------------------------------------
None
--------------------------------------------------------------------------------
None
----------------------------------------

**Итак,комментарии по второму столбцу** 

Мы имеем(2 примера):

- Место проведения занятий: у ученика (), у преподавателя (), дистанционно (например, Skype).

- Предполагаемое число занятий в неделю:  2-3.  вторник - 17, 18, 19, 20; пятница - 17, 18, 19, 20; суббота - 12, 13, 14.

- Опыт репетитора: от 5 лет.

- Возраст репетитора: от 25 лет до 65 лет.

- Пол репетитора: женский.

- Статус репетитора: школьный преподаватель, преподаватель вуза, частный преподаватель.

______________________________________________________________________________________________________________________________________


Район: г. Балашиха.

- Место проведения занятий: у преподавателя (мкр. Новый свет 10-15 минут в шаговой доступности).

- Предполагаемое число занятий в неделю: 1-2. После 17 любой день.

- Опыт репетитора: не имеет значения.

- Возраст репетитора: не имеет значения.

- Пол репетитора: женский.

- Статус репетитора: школьный преподаватель, преподаватель вуза, частный преподаватель, аспирант, студент.


**Что можно сделать** - На данном этапе не возможно,нужны полный списки от заказчика 

Опять определяем классы как я предложил выше и кодируем их от 0 до 1(в векторном диапазоне) - в моем видение(у меня мало опыта,может быть есть другие методы)

Почему так?

В статусе репитора некоторые указвыают 5,кто - то 2, то больше позиций

Векторное представление даст нам снижение размерности данных и возможность впихнуть туда все классы в из столбца СТАТУС РЕПИТОРА

Так же можно и с оптытом и возрастом

КСТАТИ ХОЧУ НАПОМНИТЬ

В ноутбуке по учителяем я УЖЕ ВЫДЕЛИЛ опыт репититора и его взраст,уверен почти на 100%,что выделенные мною признаки в тетрадки с учителями,отлично коррелировали между пожеланиями учеников!!!

Следовательно полученная метрика была бы выше(правда я еще до этого не дошёл,но уверен ЭТО БЫЛО БЫ ТАК =D )

Попробуем через регулярки

In [22]:
def extract_features(text):
    if text is None:
        return pd.Series({
            'num_sessions_weekly': None,
            'tutor_experience_years': None,
            'tutor_age_from': None
        })
    
    features = {}
    
    # Извлечение количества занятий в неделю
    sessions_match = re.search(r'\b(\d+)(?=\s*в неделю)', text)
    features['num_sessions_weekly'] = int(sessions_match.group(1)) if sessions_match else None
    
    # Извлечение опыта репетитора
    exp_match = re.search(r'опыт репетитора: (?:от\s+)?(\d+)', text, re.IGNORECASE)
    features['tutor_experience_years'] = int(exp_match.group(1)) if exp_match else None
    
    # Извлечение возраста репетитора
    age_match = re.search(r'Возраст репетитора: (?:от\s+)?(\d+)', text, re.IGNORECASE)
    features['tutor_age_from'] = int(age_match.group(1)) if age_match else None
    
    return pd.Series(features)

In [23]:
new_columns = df['add_info'].apply(extract_features)

# Перед добавлением новых столбцов удаляем старые, если они существуют
df = df.drop(columns=new_columns.columns, errors='ignore')

# Добавляем новые столбцы в DataFrame
df = df.join(new_columns)

In [24]:
df = df.drop_duplicates(subset=['order_date'])

In [25]:
# Количество заявок клиента
df['client_orders_count'] = df.groupby('client_id')['id'].transform('count')

# Средняя стоимость занятия по группе заявок
df['group_avg_lesson_price'] = df.groupby('original_order_id')['lesson_price'].transform('mean')

# Средняя запланированная длительность занятий по группе
df['group_avg_lesson_duration'] = df.groupby('original_order_id')['lesson_duration'].transform('mean')

In [26]:
# Количество уникальных клиентов в каждой группе
df['unique_clients_in_group'] = df.groupby('original_order_id')['client_id'].transform('nunique')

In [27]:
# Минимальная, максимальная занятий в каждой группе
df['group_min_lesson_price'] = df.groupby('original_order_id')['lesson_price'].transform('min')
df['group_max_lesson_price'] = df.groupby('original_order_id')['lesson_price'].transform('max')

In [28]:
# Среднее количество заявок в каждой группе
df['group_avg_lessons_per_week'] = df.groupby('original_order_id')['lessons_per_week'].transform('mean')

In [29]:
# Разница в днях заявок в каждой группе
df['date_diff_within_group'] = df.groupby('original_order_id')['order_date'].transform(lambda x: x.max() - x.min()).dt.days

In [30]:
# Разница в днях между заявками в группе
df['price_deviation_from_group_avg'] = abs(df['lesson_price'] - df['group_avg_lesson_price'])

In [31]:
# Cреднее отклонение цены в заявках группы
df['price_deviation_from_group_avg'] = abs(df['lesson_price'] - df['group_avg_lesson_price'])


In [32]:
# Процент заявок с определенной длинтельностью в группе
df['percent_of_60min_lessons_in_group'] = df.groupby('original_order_id')['lesson_duration'].transform(lambda x: (x == 60).mean() * 100)


In [33]:
# Средняя стоимость урока
df['avg_lesson_cost'] = df['lesson_price'] / df['lesson_duration']

# Процентное соотношение опыта репетитора
df['experience_range'] = df['teacher_experience_to'] - df['teacher_experience_from']

In [34]:
# Временные признаки из order_date
df['order_weekday'] = df['order_date'].dt.weekday
df['order_month'] = df['order_date'].dt.month
df['order_year'] = df['order_date'].dt.year
df['is_weekend'] = df['order_date'].dt.weekday.isin([5, 6]).astype(int)
df['day_of_year'] = df['order_date'].dt.dayofyear

In [35]:

#НЕХВАТАЕТ МОЩНОСТЕЙ




## Доступность карты
#device = "cuda" if torch.cuda.is_available() else "cpu"
#device
## Загрузка предобученной модели и передача на карту
#model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2').to(device)
#
#texts = df['purpose'].iloc[:500].tolist()
#embeddings = []
#for text in tqdm(texts, desc="Creating embeddings"):
#    embedding = model.encode(text, show_progress_bar=False)
#    embeddings.append(embedding)

In [36]:
df = df.drop(columns=['additional_status_id','estimated_fee','start_date','working_teacher_id','comments'])

In [37]:
df.reset_index(drop=True).to_feather('order_cleaned.feather')